# Association Rules
Apriori를 이용한 연관분석 실시

목적 : 연관분석을 통해 변수간 조건발생 케이스와 지표를 확인하여, 케이스별 추가 프로모션 등의 마케팅 활용 가능


### **연관규칙 결과 해석을 위한 distribution channel & market segment 변수 설명**

#### **호텔 시장 세분화**
시장 세분화에는 호텔 산업의 목표 시장을 분석하고 가격 민감도 및 예약 행동을 기반으로 세분화하는 작업이 포함됩니다. 호텔 산업의 시장 세분화는 예약 감소의 원인을 식별하는 데 도움이 됩니다. 경기 침체의 특성에 따라 고유한 영업 및 마케팅 노력이 예약을 늘리고 수익을 올리기 위해 적용됩니다. 5개의 시장 부문이 있습니다.

    - Transient – 그룹이나 회사에 연결되지 않은 게스트입니다. 최적가(BAR) 또는 공개적으로 사용 가능한 동적 요금으로 개별 예약을 하는 것입니다.
    - Corporate – 할인된 기업 요금으로 호텔에 숙박하는 고객입니다. 출장이 잦은 법인 고객은 약정으로 할인된 요금을 이용하실 수 있습니다.
    - Group – 이 고객 세그먼트는 대규모 그룹 또는 이벤트를 위해 계약된 요금으로 호텔에 숙박합니다. 요금은 요구 사항에 따라 유동적입니다.
    - Wholesale – 이 세그먼트의 객실은 여행사 또는 대량 티켓 구매 시 예약 담당자에게 할인된 가격으로 제공됩니다.
    - Other – 이 세그먼트는 무료 요금, 업계 요금, 직원이 이용할 수 있는 요금 등과 같은 다양한 범주로 구성됩니다.

#### **채널 세분화**
시장 세그먼트를 정의한 후에는 예약을 받는 방법을 기반으로 채널 세그먼트를 식별해야 합니다. 이렇게 하면 최대의 수익을 창출하도록 호텔 수익 관리 전략을 최적화할 수 있습니다. 그러나 방법에 대해 논의하기 전에 다양한 채널을 살펴보겠습니다.

    - Direct – 이 세그먼트는 전화나 이메일을 통해 호텔에 직접 예약하는 것으로 구성됩니다.
    - Website – 이 부문에서 보안 예약은 호텔 웹사이트의 호텔 예약 시스템을 통해 이루어집니다.
    - Online TA – OTA는 사람들이 플랫폼을 통해 다양한 호텔을 예약할 수 있는 온라인 타사 여행사입니다. 예를 들어 Trivago , TripAdvisor 등
    - GDS – GDS 또는 글로벌 유통 시스템은 호텔과 여행 사이트 간의 도관 역할을 하는 대규모 네트워크입니다.

    이 세분화를 기반으로 다양한 예약 패턴을 식별할 수 있습니다. 가장 많은 예약을 가져오는 채널을 식별하고 그에 따라 마케팅 및 호텔 수익 관리 전략을 변경할 수 있습니다. 스스로에게 다음과 같은 질문을 할 수 있습니다.


---

***Hotel Market Segmentation***
Market segmentation involves analyzing the target markets for the hotel industry and segmenting them based on the pricing sensitivity and booking behaviors. Market segmentation in the hotel industry helps identify the reason for a drop in the bookings. Based on the nature of the downturn, unique Sales and Marketing efforts are applied to increase bookings, hence the revenue. There are 5 market segments:

    - Transient – These are the guests who are not linked to any group or company. They are the ones that make individual bookings at your Best Available Rate (BAR) or publically available dynamic charges.
    - Corporate – These are the guests that stay at the hotel at discounted company rates. The discounted rates are available on a contract basis to corporate clients who bring in frequent business.
    - Group –This segment of the guests stays at your hotel under contracted rates for a large group or for an event. The rates are flexible depending on the requirements.
    - Wholesale –Rooms under this segment are available at discounted rates to tour operators or booking agents on buying bulk tickets.
    - Other – This segment consists of different categories such as complimentary rates, industry rates, rates available to the staff, etc.


**Channel Segmentation**
Once you define market segments, you must identify channel segments based on how you receive bookings. Doing this, you will be able to ensure that your hotel revenue management strategy is optimized to generate maximum revenue. But before we discuss how, let’s look at various channels.

    - Direct –This segment consists of direct bookings with the hotel done over phone calls or emails.
    - Website –Under this segment, secure reservations are made through a hotel reservation system on the hotel’s website.
    - OTAs – OTAs are online third-party travel agencies that allow people to make reservations for various hotels through their platform. For example, Trivago, TripAdvisor, etc.
    - GDS – GDS or global distribution system is a large network that acts as a conduit between hotels and travel sites.
  
    Based on this segmentation you can identify various booking patterns. You can identify the channel that brings in the most bookings and alter your Marketing and Hotel revenue management strategies accordingly. You can ask yourself these questions:

In [73]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [74]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

### 전처리 
1. Missing values를 0 등 적절한 값으로 대체
2. meal의 Undefined 변수는 SC타입의 MEAL임을 알아내어 대체
3. 게스트 총계가 0인 팀 제외
4. is_canceled가 0인 팀의 경우 무의미하다고 판단하여 제외
5. adr 구간화, adr_1 구간화, total guest num을 범주화

In [75]:
#
nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

#
df_clean["meal"].replace("Undefined", "SC", inplace=True)

#
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)

df_clean.drop(df_clean.index[zero_guests], inplace=True)


#
df_clean = df_clean[df_clean['is_canceled'] == 0]

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75011 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           75011 non-null  object 
 1   is_canceled                     75011 non-null  int64  
 2   lead_time                       75011 non-null  int64  
 3   arrival_date_year               75011 non-null  int64  
 4   arrival_date_month              75011 non-null  object 
 5   arrival_date_week_number        75011 non-null  int64  
 6   arrival_date_day_of_month       75011 non-null  int64  
 7   stays_in_weekend_nights         75011 non-null  int64  
 8   stays_in_week_nights            75011 non-null  int64  
 9   adults                          75011 non-null  int64  
 10  children                        75011 non-null  float64
 11  babies                          75011 non-null  int64  
 12  meal                           

In [76]:
# adr 0 이하 이상값으로 판단하여 제거
df_clean = df_clean[df_clean['adr'] > 0]

''''''
#1
df_clean = df_clean[df_clean['adults'] < 5]

#+ 2 수행을 위한 총 인원변수 추가
    #+ 10인 이상의 이상값 제거
df_clean['guestNum'] = df_clean['adults'] + df_clean['babies'] + df_clean['children']
df_clean = df_clean[df_clean['guestNum'] < 10]
#2
df_clean['adr_1'] = df_clean['adr'] // df_clean['guestNum']

    #2+
df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
df_clean = df_clean.dropna()


#3 adr_1 이 0보다 작거나 같은 경우, 무의미한 데이터로 간주하여 삭제한다. & 400이상인 경우 이상치로 간주하여 제거
df_clean = df_clean[df_clean['adr_1'] > 0]
df_clean = df_clean[df_clean['adr_1'] < 400]

''''''

# adr 구간화
labels = ['low','middle','high']
df_clean['adr_bin'] = pd.qcut(df_clean['adr'],3,labels=labels)
df_clean.head()

# adr_1 구간화
labels = ['low','middle','high']
df_clean['adr_1_bin'] = pd.qcut(df_clean['adr_1'],3,labels=labels)
df_clean.head()

# guestNum 구간화
labels_g = ['1g','couple','3g','4g','5g']
df_clean['guestNum_bin'] = pd.cut(df_clean['guestNum'],5,labels=labels_g)
df_clean.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,guestNum,adr_1,adr_bin,adr_1_bin,guestNum_bin
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,75.0,0,0,Check-Out,2015-07-02,1.0,75.0,low,high,1g
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,75.0,0,0,Check-Out,2015-07-02,1.0,75.0,low,high,1g
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,98.0,0,1,Check-Out,2015-07-03,2.0,49.0,middle,middle,couple
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,98.0,0,1,Check-Out,2015-07-03,2.0,49.0,middle,middle,couple
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,107.0,0,0,Check-Out,2015-07-03,2.0,53.0,middle,middle,couple


### 전처리 2

특정 object 변수들을 연관규칙 도출을 위해 <br>
'hotel','meal','market_segment','distribution_channel','country','arrival_date_year','customer_type','adr_bin','guestNum_bin','assigned_room_type' 변수들을 더미화

In [77]:
df_clean = pd.get_dummies(data=df_clean,
               columns=['hotel','meal','market_segment','distribution_channel','country','arrival_date_year','customer_type','adr_bin','guestNum_bin','assigned_room_type'],
               drop_first=False,
               dtype=None
               )
df_clean.columns.values

array(['is_canceled', 'lead_time', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults',
       'children', 'babies', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'guestNum',
       'adr_1', 'adr_1_bin', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC',
       'market_segment_Aviation', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_chan

## 연관규칙분석에 사용할 변수만 선정

        - 아이 포함 여부
        - 아기 포함 여부
        - 주차공간 필요 여부
        - 호텔타입
        - 주요 국가 (EDA 국가 비율 참고하여 선정): 포르투갈, 프랑스, 독일, 영국, 에스파뇰
        - 식사 타입
        - 시장세분화 타입 : 회사, 직접, 그룹
        - adr 구간 : high, middle, low
        - adr_1 구간 : high, middle, low
        - customer type : 그룹,Transient,Contract, Transient-party
        - 게스트 수 : 1~5
        - 룸타입 : A~K 

### 

In [78]:
clean_df = df_clean[[
        # 'company',
       'children', 
       'babies', 
       'required_car_parking_spaces', 
       'total_of_special_requests',
       'hotel_City Hotel', 'hotel_Resort Hotel',
       
       'country_FRA','country_PRT','country_GBR','country_ESP','country_DEU',
       
       'meal_BB', 'meal_FB','meal_HB', 'meal_SC',
        # 'market_segment_Aviation','market_segment_Complementary', 
        'market_segment_Corporate',
       'market_segment_Direct',
       'market_segment_Groups',
    #    'market_segment_Offline TA/TO', 'market_segment_Online TA',
    #    'distribution_channel_Corporate',
    #    'distribution_channel_Direct',
       'distribution_channel_GDS',
    #    , 'distribution_channel_TA/TO',
       'adr_bin_low','adr_bin_middle','adr_bin_high',
    #    'arrival_date_year_2015','arrival_date_year_2016',
        'customer_type_Group',
        'customer_type_Transient',
        'customer_type_Contract',
        'customer_type_Transient-Party',
        'guestNum_bin_1g','guestNum_bin_couple','guestNum_bin_3g','guestNum_bin_4g','guestNum_bin_5g',
        
        'assigned_room_type_A', 'assigned_room_type_B',
       'assigned_room_type_C', 'assigned_room_type_D',
       'assigned_room_type_E', 'assigned_room_type_F',
       'assigned_room_type_G', 'assigned_room_type_H',
       'assigned_room_type_I', 'assigned_room_type_K'
       ]]
clean_df.dropna()

,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,country_FRA,...,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K
2,0.0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,0.0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
5,0.0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
6,0.0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
119386,0.0,0,0,0,0,0,2,1,0,1,...,0,0,0,0,1,0,0,0,0,0
119387,0.0,0,0,0,0,0,4,1,0,0,...,0,0,0,1,0,0,0,0,0,0
119388,0.0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


### +전처리: 수치형 변수를 0 / 1 로 분류하여 저장하는 encoding을 실시

In [79]:
#
def hot_encode(x):
    if(x<= 0):
        return 0   
    if(x>= 1):
        return 1

# 
clean_encoded = clean_df.applymap(hot_encode)
clean_df = clean_encoded.copy()

# Apriori 모델 설계

최소 지지도 0.05
최소 향상도 1

In [80]:
pd.set_option('display.max_row', 500)

# 모델설계
frq_items = apriori(clean_df, min_support = 0.05, use_colnames = True)

# 데이터프레임으로 조직화 ( 향상도 최소값은 1로 설정)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(500)


# 조건절 길이와 종속절의 길이를 계산하여 파생변수로 추가
    #-> 특정 절의 길이에 따라 결과값 출력 가능
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))



# 결과 선보고 (선정)


---조건절 / 종속절 / 조건 지지도 / 종속 지지도 / **지지도 support / 신뢰도 confidence / 향상도 lift**---

### 커플은 호텔식사를 하지 않은 경우가 많았다.
- (meal_SC)	(guestNum_bin_couple)	0.099839	0.664245	**0.079697	0.798253	1.201744**
<br> =  meal_SC 타입의 식사유형을 선택했을 경우, 2인게스트(커플로 추정)일 확률이 높다.

                커플고객은 식사를 하지 않는 경우가 많음? -> 식사를 하지 않는 이유를 탐색하고 포함시킬 방법을 찾아 기대매출을 높일 방법을 탐색할 필요.***

### Transient 고객은 높은 adr을 가진다.

- (adr_bin_high)	(customer_type_Transient)	0.333329	0.705579	**0.271444	0.814342	1.154147**
<br> = adr이 높은 구간에 속하는 팀의 경우, Transient 타입의 고객이었을 확률이 높다.

        이유 : Transient 타입의 짧은일정을 보낸 게스트팀의 경우 짧은타입의 숙박에 비해 비교적 높은 금액을 지불함을 알 수 있다.

        제안 : Transient Type고객은 짧게 머물러 방의 회전율을 높여줄 뿐 아니라, adr이 높으므로 매우 높은 잠재가치를 지닌 타겟이라고 생각할 수 있다. 
        해당 타입의 고객들을 집중적으로 공략함으로써 매출액의 증대를 꾀할 수 있을 것이다.

### Corporate 예약 채널을 사용한 고객의 특징 및 제안

- (distribution_channel_Corporate)	(guestNum_bin_1g)	0.068604	0.207855	**0.050437	0.735201	3.537081**	
<br> = Corporate 예약채널을 통해 예약한 고객, 즉 법인 계약을 통해 호텔에 숙박한 고객의 경우 높은 신뢰도로 1인고객이다.

- (distribution_channel_Corporate)	(meal_BB)	0.068604	0.769318	**0.066055	0.962853	1.251567**	
<br> = 회사 채널을 통한 예약자일 경우 BB타입의 식사유형을 선택할 확률이 높다.

- (guestNum_bin_1g)	(adr_bin_low)	0.207855	0.333492	**0.109774	0.528127	1.583627**
<br> = 1인 고객의 경우 adr이 낮을 확률이 높다.

- (guestNum_bin_1g)	(adr_1_bin_high)	0.207855	0.325397	**0.144607	0.695712	2.138039**
<br> = 1인 고객의 경우 adr_1(1인당1일평균 숙박관련지출액)이 높은 구간에 속한다.

        종합 : Corporate 채널을 통한 비즈니스 숙박고객의 경우 1인 고객인 경우가 많은데, 1인고객의 경우 당연하게 총액이 작기때문에 adr이 작은 구간에 소속되었으나,
        adr_1(1인당 1일 평균 숙박관련지출액)은 높은 층에 속하므로 법인채널을 통해 예약한 1인 비즈니스 고객은 고부가가치 고객임을 유추할 수 있다. 따라서 비즈니스 숙박을 고려하는 여러 회사에 컨택 및 제휴하기 위한 전략적인 노력이 매출액의 효율적 상승에 도움을 줄 수 있어 보인다.
---
### 아이 동반 게스트팀의 특징 및 제안

- (children)	(adr_bin_high)	0.073319	0.333329	**0.053109	0.724349	2.173078**
<br> = 아이를 동반한 팀의 경우 adr이 높은 구간에 속할 확률이 높다.

- (children)	(customer_type_Transient)	0.073319	0.705579	**0.064829	0.884201	1.253156**
<br> = 아이들을 동반한 팀의 경우 Transient타입의 고객일 확률이 높다.


        종합 : 게스트팀이 아이를 동반했을 경우, 많은 경우 adr이 높은 구간에 속해있었음을 알 수 있으며, Transient타입의 고객인 경우도 많은 것을 알 수 있다.
        아이를 동반한 경우 회전율이 빠르고 높은 adr을 지닌 잠재가치가 높은 타겟층으로 해석되므로, 
        
        1. 부모 & 아이들을 위한 편의시설 혹은 놀이시설을 확충
        2. 아이 동반 팀을 타겟으로 프로모션 실시
        
        등의 방법을 통해 높은 잠재가치를 지닌 아이동반 가족단위의 게스트를 많이 확보하는 등의 전략을 고려해볼 수 있다.

---
### 3인 게스트팀의 특징

- (guestNum_bin_3g)	(customer_type_Transient)	0.095901	0.705579	0.082191	0.857041
<br> = 3인의 게스트팀일 경우 Transient 고객타입일 확률이 높다.	

- (guestNum_bin_3g)	(adr_bin_high)	0.095901	0.333329	**0.069067	0.720193	2.160609**
<br> = Guest 수가 3인일 경우 adr이 높은 구간에 속할 확률이 높다.

- (guestNum_bin_3g)	(customer_type_Transient, hotel_City Hotel)	0.095901	0.428984	**0.059078	0.616030	1.436019**
<br> => 3인 팀의 경우, Transient 타입이며 시티 호텔에 머무를 확률이 높다.

- (customer_type_Transient, guestNum_bin_3g)	(adr_bin_high)	0.082191	0.333329	**0.062008	0.754435	2.263337**
<br> => 3인 팀이며, Transient 고객 타입일 경우, adr이 높은 구간에 속할 확률이 lift 2.26으로 높다.

- (required_car_parking_spaces)	(customer_type_Transient)	0.099512	0.705579	0.085993	0.864147
<br> = 주차공간을 필요로 할 경우 Transient(임시) 고객타입일 확률이 높다.

- (required_car_parking_spaces)	(total_of_special_requests)	0.099512	0.513287	**0.057115	0.573952	1.118189**
<br>-> 주차공간을 필요로 했을 경우, 특별 주문이 존재했을 확률이 높다.

- (guestNum_bin_3g)	(total_of_special_requests)	0.095901	0.513287	0.069326	0.722893
<br> = Guest 수가 3인에 속할 경우 특별 요청을 할 확률이 높다.


        종합 : Transient에 adr가 높기 때문에 고잠재가치 타겟이지만, 특별한 제안점은 떠오르지 않는다.


---
### 주요 국가별 게스트 팀 특징 파악

- (country_GBR)	(hotel_Resort Hotel)    **0.079915	0.611024	1.58626**
<br> = 영국에서 온 팀의 경우 리조트 호텔을 선택할 확률이 높다.

- (guestNum_bin_couple, country_GBR)	(hotel_Resort Hotel)	0.099008	0.385197	**0.062089	0.627116	1.628040**
<br> => 영국에서 온 커플(2인)팀인 경우, Resort hotel을 선택할 확률이 더 높다.

- (country_FRA)	(hotel_City Hotel)	0.114844	0.614803	**0.095955	0.835529	1.359019**
<br> = 프랑스에서 온 팀의 경우 시티 호텔을 선택할 확률이 높다.

- (country_FRA)	(total_of_special_requests)	0.114844	0.513287	**0.063847	0.555951	1.083119**
<br> = 프랑스에서 온 팀의 경우 특별 요청이 존재했을 확률이 높다.

- (country_DEU)	(hotel_City Hotel)	0.082136	0.614803	**0.067895	0.826614	1.344518**	
<br> = 독일에서 온 팀의 경우 시티 호텔을 선택할 확률이 높다.

- (adr_bin_high)	(hotel_City Hotel)	0.333329	0.614803	**0.228311	0.684942	1.114084**
<br> = adr이 높은 구간에 속했을 경우 city hotel에 머물렀을 확률이 높다.

- (hotel_Resort Hotel)	(adr_bin_low)	0.385197	0.333492	**0.206942	0.537237	1.610942**
<br> = 리조트 호텔을 선택했을 경우 adr이 낮은 구간에 속할 확률이 높다.

<br>
<br>

    종합 : 주요 국가별로 살펴보았을 때, 영국은 리조트 호텔을 프랑스,독일 출신의 주요국가들은 시티 호텔을 선호하는 경향이 보인다.
    adr이 높은 구간에 속했을 경우 city호텔에, 리조트 호텔에 속했을 경우 adr이 낮은 구간에 속했던 규칙을 함께 참고해볼 때, 프랑스 독일에 city호텔 위주의 프로모션을 실시하여 잠재가치가 높은 고객들을 많이 유치할 수 있을 것이다. 마찬가지로, 영국 고객들에겐 리조트 호텔 위주의 프로모션을 더욱 활발하게 실시하여 더욱 효과적인 마케팅 성과를 달성할 수 있을 것이다.

    특히 프랑스 팀의 경우 특별 요청이 특히 많았던 점을 미루어볼 때, 해당 국가에서 온 게스트팀에 특히 섬세한 서비스가 요구된다는 것을 알 수 있다. 해당 데이터에 특별요청의 요구사항이 나와있지 않기 때문에 해당 연구에서 더 자세히 알아볼 수는 없겠지만 사전에 파악하여 섬세한 서비스를 제공할 경우 고잠재가치를 지닌 프랑스 고객의 확보에 특히 차별성을 더할 수 있을 것이라고 생각된다.

---
- (required_car_parking_spaces)	(hotel_Resort Hotel)	0.099512	0.385197	**0.073946	0.743084	1.929100**
<br> = 주차공간이 필요한 경우 리조트 호텔을 많이 선택하였다? -> 선후관계 틀린듯 , 차량으로 이동하여 리조트호텔이 있는 관광지로 가기때문에 주차공간이 비교적 많이 소요된것으로 보임


- (market_segment_Groups)	(adr_bin_low)	0.101965	0.333492	**0.059705	0.585539	1.755778**
<br> = 시장세분화가 그룹에 속할 경우 adr이 낮은 구간에 속할 확률이 높다.



### 조건절의 길이가 1, 종속절의 길이가 1인 규칙들 출력 및 확인

In [81]:

rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 1) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.)  
    ]

rules_filtered1.sort_values(by='lift',ascending=False).head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
145,(market_segment_Groups),(customer_type_Transient-Party),0.101965,0.249571,0.096677,0.948142,3.799092,0.071230,14.470907,1,1
2,(children),(adr_bin_high),0.073319,0.333329,0.053109,0.724349,2.173078,0.028669,2.418537,1,1
164,(guestNum_bin_3g),(adr_bin_high),0.095901,0.333329,0.069067,0.720193,2.160609,0.037101,2.382613,1,1
21,(required_car_parking_spaces),(hotel_Resort Hotel),0.099512,0.385197,0.073946,0.743084,1.929100,0.035614,2.393012,1,1
79,(assigned_room_type_E),(hotel_Resort Hotel),0.078157,0.385197,0.056543,0.723452,1.878135,0.026437,2.223136,1,1
142,(market_segment_Groups),(adr_bin_low),0.101965,0.333492,0.059705,0.585539,1.755778,0.025700,1.608130,1,1
70,(hotel_Resort Hotel),(adr_bin_low),0.385197,0.333492,0.206942,0.537237,1.610942,0.078482,1.440278,1,1
71,(adr_bin_low),(hotel_Resort Hotel),0.333492,0.385197,0.206942,0.620530,1.610942,0.078482,1.620164,1,1
65,(country_GBR),(hotel_Resort Hotel),0.130789,0.385197,0.079915,0.611024,1.586264,0.029536,1.580569,1,1
150,(guestNum_bin_1g),(adr_bin_low),0.207855,0.333492,0.109774,0.528127,1.583627,0.040456,1.412474,1,1


### 조건절의 길이가 1이며 종속절의 길이가 2인 규칙들 확인
**+설정: 지지도 0.05이상, 신뢰도 0.5이상, 향상도 1.15이상**

In [82]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 2) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.15)  
    ]


rules_filtered_12 =rules_filtered1.sort_values(by='lift', ascending=False)

rules_filtered_12.drop(['antecedent_len','consequent_len'],axis=1)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1070,(market_segment_Groups),"(customer_type_Transient-Party, adr_bin_low)",0.101965,0.104132,0.056175,0.550922,5.290613,0.045557,1.994906
1081,(market_segment_Groups),"(customer_type_Transient-Party, assigned_room_...",0.101965,0.167557,0.067636,0.663325,3.958803,0.050551,2.472544
1075,(market_segment_Groups),"(customer_type_Transient-Party, guestNum_bin_c...",0.101965,0.160334,0.063861,0.626303,3.906236,0.047513,2.246917
935,(market_segment_Groups),"(customer_type_Transient-Party, meal_BB)",0.101965,0.177506,0.066287,0.650094,3.662385,0.048188,2.350613
567,(market_segment_Groups),"(customer_type_Transient-Party, hotel_City Hotel)",0.101965,0.166208,0.055044,0.539829,3.247915,0.038096,1.811918
419,(guestNum_bin_3g),"(total_of_special_requests, adr_bin_high)",0.095901,0.217763,0.051473,0.536734,2.464766,0.030590,1.688529
988,(guestNum_bin_3g),"(meal_BB, adr_bin_high)",0.095901,0.244106,0.056079,0.584766,2.395544,0.032670,1.820406
1148,(guestNum_bin_3g),"(customer_type_Transient, adr_bin_high)",0.095901,0.271444,0.062008,0.646582,2.382012,0.035976,2.061460
612,(guestNum_bin_3g),"(hotel_City Hotel, adr_bin_high)",0.095901,0.228311,0.051378,0.535740,2.346535,0.029483,1.662190
239,(required_car_parking_spaces),"(hotel_Resort Hotel, customer_type_Transient)",0.099512,0.276595,0.063766,0.640783,2.316683,0.036241,2.013840


### 조건절의 길이가 1보다 크며 종속절의 길이가 1인 규칙들 확인
**+설정: 조건절 4개 이상, 지지도 0.05이상, 신뢰도 0.5이상, 향상도 1.5이상**

In [90]:
rules_filtered2 = rules[
    (rules['antecedent_len'] > 3) &
    # (rules['consequent_len'] > 3) &
    (rules['support'] > 0.05) &
    (rules['confidence'] > 0.5) &
    (rules['lift'] > 1.0)  
    ]

rules_filtered2.sort_values(by='lift', ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
3834,"(hotel_Resort Hotel, customer_type_Transient, ...",(adr_bin_low),0.078797,0.333492,0.050342,0.638879,1.915724,0.024064,1.845664,4,1
3782,"(country_PRT, customer_type_Transient, meal_BB...",(hotel_Resort Hotel),0.076494,0.385197,0.055957,0.731516,1.899069,0.026491,2.289906,4,1
3187,"(customer_type_Transient, total_of_special_req...",(hotel_Resort Hotel),0.081659,0.385197,0.059582,0.729640,1.894197,0.028127,2.274011,4,1
3778,"(hotel_Resort Hotel, customer_type_Transient, ...",(adr_bin_low),0.088787,0.333492,0.055957,0.630238,1.889812,0.026347,1.802531,4,1
4007,"(adr_bin_middle, meal_BB, assigned_room_type_A...","(guestNum_bin_couple, hotel_City Hotel)",0.080978,0.399820,0.059309,0.732413,1.831857,0.026933,2.242936,4,2
4158,"(adr_bin_middle, customer_type_Transient, assi...","(guestNum_bin_couple, hotel_City Hotel)",0.083472,0.399820,0.060958,0.730286,1.826536,0.027585,2.225243,4,2
3211,"(guestNum_bin_couple, total_of_special_request...",(hotel_Resort Hotel),0.080228,0.385197,0.056325,0.702055,1.822587,0.025421,2.063480,4,1
3811,"(guestNum_bin_couple, customer_type_Transient,...",(hotel_Resort Hotel),0.117869,0.385197,0.081959,0.695341,1.805155,0.036556,2.018000,4,1
3611,"(customer_type_Transient, hotel_City Hotel, as...",(adr_bin_high),0.103451,0.333329,0.059268,0.572915,1.718769,0.024785,1.560981,4,1
3702,"(meal_SC, guestNum_bin_couple, hotel_City Hote...",(assigned_room_type_A),0.065646,0.550437,0.060277,0.918206,1.668139,0.024143,5.496301,4,1


### 조건절의 길이가 2인 규칙들 확인
**+설정: 지지도 0.05이상, 신뢰도 0.5이상, 향상도 1.2이상**

In [84]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 1.2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
265,"(meal_SC, total_of_special_requests)",(hotel_City Hotel),0.059882,0.614803,0.058069,0.969731,1.577305,0.021254,12.725989,2,1
544,"(meal_SC, customer_type_Transient)",(hotel_City Hotel),0.083363,0.614803,0.080596,0.966814,1.572559,0.029345,11.607149,2,1
565,"(hotel_City Hotel, market_segment_Groups)",(customer_type_Transient-Party),0.057524,0.249571,0.055044,0.956882,3.834113,0.040687,17.404187,2,1
1078,"(assigned_room_type_A, market_segment_Groups)",(customer_type_Transient-Party),0.070730,0.249571,0.067636,0.956262,3.831628,0.049984,17.157392,2,1
1073,"(guestNum_bin_couple, market_segment_Groups)",(customer_type_Transient-Party),0.066818,0.249571,0.063861,0.955741,3.829541,0.047185,16.955552,2,1
933,"(meal_BB, market_segment_Groups)",(customer_type_Transient-Party),0.070389,0.249571,0.066287,0.941723,3.773372,0.048720,12.876968,2,1
1068,"(market_segment_Groups, adr_bin_low)",(customer_type_Transient-Party),0.059705,0.249571,0.056175,0.940881,3.769998,0.041274,12.693555,2,1
582,"(guestNum_bin_1g, adr_bin_middle)",(hotel_City Hotel),0.064543,0.614803,0.059282,0.918497,1.493969,0.019601,4.726150,2,1
545,"(meal_SC, hotel_City Hotel)",(customer_type_Transient),0.088010,0.705579,0.080596,0.915763,1.297889,0.018498,3.495163,2,1
1054,"(market_segment_Direct, adr_bin_high)",(customer_type_Transient),0.063084,0.705579,0.057701,0.914668,1.296337,0.013190,3.450312,2,1
